In [ ]:
import eumdac
import datetime
import shutil
from datetime import timedelta
from sentinelsat import read_geojson, geojson_to_wkt
import zipfile
import gzip
import os
import pandas as pd
from glob import glob

download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

In [ ]:
def get_datastore(consumer_key, consumer_secret):
    token = eumdac.AccessToken((consumer_key, consumer_secret))
    print(f"This token '{token}' expires {token.expiration}")
    return eumdac.DataStore(token)

def Sentinel_request(datastore, roi,
                     collectionID='EO:EUM:DAT:0409',
#                      geojson_file, 
                     opath = './',
                     start_date = None, end_date = None):

    WKT = 'POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi]))
    selected_collection = datastore.get_collection(collectionID)
    print(selected_collection.title)
    
    if(end_date == None):
        end_date = datetime.date.today()
    if(start_date == None):
        start_date = end_date-timedelta(days=1)
        
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    print('Searching from %s to %s' %(start_date, end_date))
    products = selected_collection.search(geo=WKT, 
                                          dtstart=start_date, 
                                          dtend=end_date)
    print(f'Found Datasets: {len(products)} datasets for the given time range')
    for product in products:
        with product.open() as fsrc, open(opath+fsrc.name, mode='wb') as fdst:
#             if os.path.isfile(fdst.name):
            if os.path.getsize(fdst.name)>0:

                print(f'Product {fdst.name} exists.')
            else:
                shutil.copyfileobj(fsrc, fdst)
                print(f'Download of product {product} finished.')
    print('All downloads are finished.')
            
    rpath = opath+'S3_%s_%s' % (start_date.strftime("%Y%m%d"),
                                start_date.strftime("%Y%m%d"))                    
    for f in glob(opath+'S3*.zip'):
        try:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                print('Unziping %s' %f)
                zip_ref.extractall(rpath)
                os.remove(f)
        except:
            print('Bad zip file: %s' %f)

    print(f'Done! Files in {rpath}')

In [3]:
ls

GOMEX_FB.geojson  Sentinel DL 2.ipynb  Untitled.ipynb  products/
LakeO.geojson     Sentinel DL.ipynb    flbay.geojson
README.md         Sentinel Vis.ipynb   gomex.geojson


In [ ]:
consumer_key = 'yawsJIqGoAetBf_MqpcByUwnrWMa'
consumer_secret = 'w6nWuFAtEgPhFESfK7Q4MzOs9t0a'

roi = [[-81.0955810546875,26.675685969067487],
       [-80.54489135742188,26.675685969067487],
       [-80.54489135742188,27.243641579169292],
       [-81.0955810546875,27.243641579169292],
       [-81.0955810546875,26.675685969067487]]

datastore = get_datastore(consumer_key, consumer_secret)
Sentinel_request(datastore, roi, opath='./products/', start_date=datetime.date(2022,3,23),
                 end_date=datetime.date(2022,3,24))

In [ ]:
consumer_key = 'yawsJIqGoAetBf_MqpcByUwnrWMa'
consumer_secret = 'w6nWuFAtEgPhFESfK7Q4MzOs9t0a'

roi = [[-93.97705078125,24.86650252692691],
       [-79.25537109375,24.86650252692691],
       [-79.25537109375,31.372399104880525],
       [-93.97705078125,31.372399104880525],
       [-93.97705078125,24.86650252692691]]

datastore = get_datastore(consumer_key, consumer_secret)
Sentinel_request(datastore, roi, opath='./products/', start_date=datetime.date(2022,3,23),
                 end_date=datetime.date(2022,3,24))